# PS 88 Lab 8 - Pandas, Correlation, Regression

## Part 1. Pandas and Data Science
For the rest of the course, we'll often rely on the pandas library to manage tables. In addition to being the current industry standard for data manipulation, pandas works better with some other libraries we will use in our later work with regression and plotting. The goal of the first part of this lab is to showcase the primary differences between datascience and pandas. If you notice similarities between the two libraries' approaches, it's because datascience is actually built on top of pandas! To start, load in both libraries:

In [ ]:
from datascience import Table
import pandas as pd
%matplotlib inline

How do we create a table? The process is similar, although in pandas the syntax is a bit different (don't worry about the details here; we will almost always just be importing tables from files).

In [ ]:
t = Table().with_columns([
       'letter', ['a', 'b', 'c', 'z'],
       'count',  [  9,   3,   3,   1],
       'points', [  1,   2,   2,  10],
   ])
t

In [ ]:
pd_t = pd.DataFrame({
        'letter': ['a', 'b', 'c', 'z'],
        'count':  [  9,   3,   3,   1],
        'points': [  1,   2,   2,  10]
        })
pd_t

For the rest of the lab, we will work with data from ProPublica about members of the Senate. To begin, notice that reading in a CSV is remarkably similar.

In [ ]:
# datascience Table version
senator_table = Table.read_table('data/members.csv')
senator_table

In [ ]:
# pandas version
senator_df = pd.read_csv('data/members.csv')
senator_df

Accessing a single column has two separate methods in datascience. The first, `.column`, returns an array of the values. The second, `.select`, returns the actual column as a miniature table. Meanwhile, pandas selection returns a single list of the values.

In [ ]:
senator_table.column('twitter_account')

In [ ]:
senator_table.select('twitter_account')

The main way to do this in pandas is to put square brackets after the name of the data frame, with the name of the variable in quotation marks.

In [ ]:
senator_df['twitter_account']

**Question 1.1. Write code to pull the column which indicates the party of the Senator using pandas.**

In [ ]:
# Code for 1.1

If we want to access all of a row, datascience has the `.take()` function, and pandas has the `.loc` function (note the square brackets)

In [ ]:
senator_table.take(0)

In [ ]:
senator_df.loc[0]

Relabeling columns is similar in both libraries, although pandas requires specifying that you're changing the columns.

In [ ]:
senator_table.relabeled('facebook_account', 'fb')

In [ ]:
senator_df.rename(columns={"facebook_account":'fb'})

Creating a filtered version of the data requires the `.where` method in datascience. In pandas, it is similar to previous accessors, albeit with the condition added rather than the specific row or column value.

In [ ]:
senator_table.where('gender', 'M')

In [ ]:
senator_df[senator_df['gender'] == 'M']

**Question 1.2. Write code to create separate pandas data frames for Senators who are Democrats (call this `senator_D`) and Senators who are Republicans (call this `senator_R`)**

In [ ]:
# Code for 1.2

You'll notice that for more complicated filters, datascience quickly starts to look like pandas. In this case, we create a subset of the table that contains only senators who voted with their party more than 96% of the time

In [ ]:
senator_table.where(senator_table.column('votes_with_party_pct') > 98)

In [ ]:
senator_df[senator_df['votes_with_party_pct'] > 98]

To sort a datascience table, use the column you wish to sort by and the optional `descending` value. The only difference between the two libraries is that datascience uses `descending` while pandas uses `ascending` to differentiate sorting method.

In [ ]:
senator_table.sort('votes_with_party_pct')

In [ ]:
senator_table.sort('votes_with_party_pct', descending=True)

In [ ]:
senator_df.sort_values('votes_with_party_pct')

In [ ]:
senator_df.sort_values('votes_with_party_pct', ascending=False)

We can plot a histogram of a column in the datascience table fairly easily:

In [ ]:
senator_table.hist('votes_with_party_pct')

With pandas, the easiest method is to select the specifc column, and then call the `hist` function.

In [ ]:
senator_df['votes_with_party_pct'].hist()

**Question 1.3. Using pandas, create separate histograms of the `votes_with_party_pct` variable for Republicans and Democrats. (Hint: there are a few ways to do this; one is to use the separate data frames for each party from 1.2 and then use `.hist()` on that.**

In [ ]:
#Code for 1.3

## Part 2: Correlation
Our discussion of healthcare so far had a bit of a US-centric focus. Let's take a more *comparative* approach, by checking if countries that spend more on health care tend to have healthier citizens. In the following example, will look at the correlation between health care spending as a proportion of GDP and life expectancy in OECD countries.

The variable we will use for health care spending is the amount spent as a proportion of the total economic output (GDP) in 2015. 

The variable we will used to measure the health of citizens is their life expectancy, or how long  the typical person lives, also in 2015.

Since both variables are continuous, one natural way to measure the strength of the relationship is with the *correlation*: the degree to which one of the variable's change in value coincides with a similar change in value in the other variable. 

First, we need to load in the data set from a CSV. Just to see another example of how to load data, this one is stored on my website.

In [ ]:
healthspend = pd.read_csv("http://andrewtlittle.com/ps3data/outspend2015.csv")
healthspend

One way to make a scatter plot is with the `.scatterplot` function in the seaborn library. The first argument is our x axis variable, the second is the y axis variable, and the third tells us that both of these are in our `healthspend` data frame.

First we will import seaborn and some other libraries we will need.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import numpy as np
from ipywidgets import *
from scipy import stats
from IPython.display import display, Markdown

In [ ]:
sns.scatterplot(x='Spending', y='Expectancy', data=healthspend)

**Question 2.1. Create a scatterplot with life expectancy on the x axis and spending on the y axis.**

In [ ]:
# Code for 2.1

To get a sense of which "quadrant" each country is in, we can make a vertical line at the average spending and a horizontal line at the average life expectancy.

In [ ]:
sns.scatterplot(x='Spending', y='Expectancy', data=healthspend)
plt.axvline(np.mean(healthspend['Spending']))
plt.axhline(np.mean(healthspend['Expectancy']))

One way to compute the correlation is with the `.pearsonr` function in the stats library. The first number returned is the correlation. The second relates to a hypothesis test which we won't discuss yet, so you can ignore it for now.

In [ ]:
stats.pearsonr(healthspend['Spending'], healthspend['Expectancy'])

The correlation is postive at .42.

**Question 2.2. Create a dataframe called `spend_noUS` which drops out the United States. Compute the correlation between health care spending and life expectancy for this set of countries Does it go up or down? Why**

In [ ]:
# Code for 2.2

*Words for 2.2*

## Part 3: Bivariate Regression
When we have two continuous variables (one dependent, one independent), we can use *bivariate regression* to determine how closely the two are related. Biviarate regression is used to determine how changes in one variable -- the independent variable, often denoted $X$ -- can predict changes in another, the dependent variable, often denoted $Y$. Bivariate regression relies on a linear model, which follows the form $Y_i= a + b X_i$, where $a$ is the y-intercept and $b$ is the slope. 

If we assume that the relationship between our variables is not perfect (or, in the real world, if there is some predictable inaccuracy in our measurement), we add an error term $e$: $Y_i= a + b X_i + e_i$. 

To understand how we might create an equation for two variables, let's keep using the example of the relationship between health care spending and life expectancy

Above, it appears that as Spending increases (as we move further to the right on the x-axis), Expectancy also increases. If we wanted to use this data to make predictions (perhaps for other countries, or the same countries in the future), we could use a linear model to represent the variables' relationship. Below, you can change the slope and intercept of the line to best fit the data:

In [ ]:
def draw_line(slope, intercept):
    #The Linear Model
    #    def f(x):
    #    return intercept*(slope-1)/30*x +intercept
    def f(x):
        return intercept + slope*x
    x = np.arange(4,16)
    y_pred = f(x)
    points = (zip(spend_noUS.Spending, spend_noUS.Expectancy))
    #The line
    sns.scatterplot(x='Spending', y='Expectancy', data=spend_noUS)
    plt.plot(x,y_pred)
    #The actual data 
    plt.xlabel('Spending')
    plt.ylabel('expectancy')
    display(Markdown(rf'$\hat y$= {slope}$X$ + {intercept}:'))



In [ ]:
draw_line(slope=.3, intercept=75)

That line is definitely too low to be making good predictions. Let's try to do better.

**Question 3.1. Find a line that you think fits the data best by playing around with the slope and intercept arguments in the `draw_line` function.**

In [ ]:
# Code for 3.1

### What line is best?
When we are evaulating how "good" a line is, we must address the *residuals*, the difference between the real and predicted values of y: $e_i = Y_i - \hat{Y_i}$. Because every real y value has an associated residual, we need some way to aggregate the residuals if we are to measure the overall quality of a line

The main measurement of aggregate error is the the *sum of squared errors*, calculated by adding the squared values of the residuals:
$$\sum_{i=1}^n e_i^2 = \sum_{i=1}^n (Y_i-\hat{Y_i})^2$$

We want the line that results in the smallest value (indicating that the total difference between the predicted and actual values is small). Below, try to minimize either the absolute or squared loss:

In [ ]:
from IPython.display import display, Markdown

In [ ]:
def draw_line(slope, intercept):
    #The Linear Model
    def f(x):
        return intercept + slope*x
    x = np.arange(4,16)
    y_pred = f(x)
    display(Markdown(rf'$\hat y$= {slope}$X$ + {intercept}:'))
    #The line
    plt.plot(x,y_pred)
    #The Data
    sns.scatterplot(x='Spending', y='Expectancy', data=spend_noUS)
    #Print the loss
    print("Square Residual Sum:", sum([(y-f(x))**2 for x,y in zip(spend_noUS.Spending, spend_noUS.Expectancy)]))
 

In [ ]:
draw_line(slope=.3, intercept = 78)

**Question 3.2. Use the new version of `draw_line` to find a slope and intercept which give a lower Square Residual Sum than your answer in 3.1.**

Now let's us a theoretical formula to find the line that minimizes the squared residuals. To find the slope ($b$) and y-intercept ($a$), the following equations are used:
$$b = \frac{\sum_{i=1}^n (X_i - \overline{X})(Y_i - \overline{Y})}{\sum_{i=1}^n (X_i - \overline{X})^2}$$
$$a = \overline{Y}-b\overline{X}$$
*Reminder*: $\overline{X}$ represents the mean value of X.

The main way we will find this least squares line (and do later work with regression) is using the "statsmodelsformula" library, which we imported as `smf`. A nice thing about this library  is that it mimics the syntax of R, so if you end up using that language in another class or setting you will be able to translate pretty quickly. 

There are two steps to getting regression output. First, we *fit a model*, with the following syntax:

`name = smf.ols('DV ~ IV', data=df).fit()`

We can pick whatever we want for name. For DV, put the column name of the dependent variable, and for IV put the column name of the independent variable. For df, put the name of the data frame. 

Next we run a separate function which tells us to summarize the fitted model, with:

`name.summary()`

For example, this creates the best fit line for predicting life expectancy from spending, excluding the US:



In [ ]:
spend_ols = smf.ols('Expectancy ~ Spending', data=spend_noUS).fit()
spend_ols.summary()

There is lots of output here, most of which we haven't covered yet/won't cover at all. For now, the key thing to notice are the two valus ine the "coef" column in the second table. The first one gives the intercept, and the second the slope on the best fit line. How close are these to your answer from 3.2?

**Question 3.3. Find the best fit line for predicting life expectancy from health care spending using the data which also includes the US. How does the slope change, and why?**

**Question 3.4. Finally, let's return to the Senator data. Make a scatter plot with the number of votes missed on the x axis and the percentage of votes with the party on the y axis. Then find the best fit line for this relationship using `smf.ols`. Interpret the slope on missed votes.** 

*Words for 3.4.*